In [8]:
!qstat -u ho_kezau83


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
10717906                ho_kezau83  smp      dram.sh             --      1     20     200gb  24:00:00 C       -- 


In [7]:
!qdel 10717906

# Bash function

In [1]:
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  #!rm $script
  
ws = '/beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0'

#Create list of file indexes to access it later
import os
!rm -r Raw_data/.ipynb_checkpoints
reads = sorted(list(set([f.split('_')[0].split('.')[-1] for f in os.listdir('Raw_data')])))
done = sorted(list(set([f.split('_')[0].split('.')[-1] for f in os.listdir('co_assembly/NoAdapt_reads')])))
print('Raw reads:', reads)
print('Processed:', done)
print('Cleaning is missing for (if empty all good!): ', [r for r in reads if r not in done])

rm: cannot remove 'Raw_data/.ipynb_checkpoints': No such file or directory
Raw reads: ['C15A', 'C15B', 'C15C', 'C2A', 'C2B', 'C2C', 'C40A', 'C40B', 'C40C', 'C4A', 'C4B', 'C4C', 'C8A', 'C8B', 'C8C', 'T0A', 'T0B', 'T0C']
Processed: ['C15', 'C2', 'C4', 'C40', 'C8', 'T0', 'ipynb']
Cleaning is missing for (if empty all good!):  ['C15A', 'C15B', 'C15C', 'C2A', 'C2B', 'C2C', 'C40A', 'C40B', 'C40C', 'C4A', 'C4B', 'C4C', 'C8A', 'C8B', 'C8C', 'T0A', 'T0B', 'T0C']


# Clean sequences

In [ ]:
# remove sequencing artifacts, adapters, merge reads and QC
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=02:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
module load bio/bbtools/38.26 
bbduk.sh in1=%s in2=%s out1=%s out2=%s outm=%s k=31 ftm=5 ref=%s t=12
bbduk.sh in1=%s in2=%s out1=%s out2=%s ref=%s ktrim=r k=23 mink=11 hdist=1 tbo=t qtrim=r trimq=20 t=12
'''
nA = ws+'/co_assembly/NoArt_reads'
na = ws+'/co_assembly/NoAdapt_reads'
me = ws+'/co_assembly/Merged_reads'
QC = ws+'/co_assembly/QCreads'
rr = ws+'/merge_data'
#!mkdir $nA $na $me $QC
import os
for read in reads:
  if read in done: continue
  in1  = rr+'/%s_1.fq.gz'           % read
  in2  = rr+'/%s_2.fq.gz'           % read
  om   = nA+'/%s_matched.fq.gz'     % read
  o1   = nA+'/%s_unmatched_1.fq.gz' % read
  o2   = nA+'/%s_unmatched_2.fq.gz' % read
  o3   = na+'/%s_trimmed_1.fq.gz'   % read
  o4   = na+'/%s_trimmed_2.fq.gz'   % read
  r1  = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/bbtools/sequencing_artifacts.fa.gz'
  r2  = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/bbtools/adapters.fa'
  #if os.path.exists(o4): continue
  bash(string%(in1,in2,o1,o2,om,r1,o1,o2,o3,o4,r2),
       'BBTools_%s'%read,'short')

# Taxonomy on read with Kaiju (All)

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate kaiju
kaiju -z 14 -t %s -f %s -i %s -j %s -o %s
conda deactivate
'''
conda_path = '/beegfs/work/workspace/ws/ho_kezau83-conda-0'
nodes = conda_path+'/kaijudb/nodes.dmp'
db = conda_path+'/kaijudb/kaiju_db_refseq.fmi'

!mkdir kaiju_tax

for read in reads:
    file_1 = ws+'/NoAdapt_reads/%s_trimmed_1.fastq' % read
    file_2 = ws+'/NoAdapt_reads/%s_trimmed_2.fastq' % read
    out = ws+'/kaiju_tax/%s.out' % read
    if os.path.exists(out): continue
    bash(string%(nodes, db, file_1, file_2, out),'%s_kaiju'%read,'short')

In [ ]:
# Merge samples and add taxa names
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate kaiju
cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/kaiju_tax/
kaiju2table -l superkingdom,phylum,class,order,family,genus,species -t %s -n %s -r species -c 10 -o %s T0A.out T0B.out T0C.out C2A.out C2B.out C2C.out C4A.out C4B.out C4C.out C8A.out C8B.out C8C.out C15A.out C15B.out C15C.out C40A.out C40B.out C40C.out
'''
conda_path = '/beegfs/work/workspace/ws/ho_kezau83-conda-0'
nodes = conda_path+'/kaijudb/nodes.dmp'
names = conda_path+'/kaijudb/names.dmp'
out = ws+'/kaiju_tax/merge_samples.tsv'

bash(string%(nodes, names, out),'kaiju','short')

# Taxonomy on read with Kaiju (Viruses)

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate kaiju
kaiju -z 14 -t %s -f %s -i %s -j %s -o %s
conda deactivate
'''
conda_path = '/beegfs/work/workspace/ws/ho_kezau83-conda-0'
nodes = conda_path+'/kaijudb_viruses/nodes.dmp'
db = conda_path+'/kaijudb_viruses/kaiju_db_viruses.fmi'

!mkdir kaiju_tax_viruses

for read in reads:
    file_1 = ws+'/NoAdapt_reads/%s_trimmed_1.fq' % read
    file_2 = ws+'/NoAdapt_reads/%s_trimmed_2.fq' % read
    out = ws+'/kaiju_tax_viruses/%s.out' % read
    if os.path.exists(out): continue
    bash(string%(nodes, db, file_1, file_2, out),'%s_kaiju'%read,'short')

In [ ]:
# Merge samples and add virus names
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate kaiju
cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/kaiju_tax_viruses/
kaiju2table -e -l superkingdom,phylum,class,order,family,genus,species -t %s -n %s -r species -c 10 -o %s C2A.out C40A.out C40C.out C8C.out T0C.out
'''
conda_path = '/beegfs/work/workspace/ws/ho_kezau83-conda-0'
nodes = conda_path+'/kaijudb_viruses/nodes.dmp'
names = conda_path+'/kaijudb_viruses/names.dmp'
out = ws+'/kaiju_tax_viruses/merge_samples_virus.tsv'

bash(string%(nodes, names, out),'kaiju','short')

In [ ]:
# Run KrakenUniq
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate krakenuniq

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/kraken

krakenuniq --preload-size 60G --db %s --threads 14 --report-file REPORTFILE.tsv --paired %s %s > READCLASSIFICATION.tsv

'''

kraken_db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/kraken_db'
read_1 = ws+'/NoAdapt_reads/C40A_trimmed_1.fastq'
read_2 = ws+'/NoAdapt_reads/C40A_trimmed_2.fastq'

bash(string%(kraken_db, read_1, read_2),'kraken','short')

In [ ]:
# Run KrakenUniq
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate krakenuniq

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/kraken

krakenuniq-report --db %s %s > taxonomy.tsv
'''

kraken_db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/kraken_db'
kraken_file = ws+'/kraken/READCLASSIFICATION.tsv'

bash(string%(kraken_db, kraken_file ),'kraken','short')

# Assemble contigs

In [ ]:
# Assembly with metaspades and Quast
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash
module load bio/spades/3.15.0
spades.py -k 21,33,55,77,99,127 -1 %s -2 %s --meta -o %s -m 200 -t 20 
'''
na = ws+'/co_assembly/NoAdapt_reads'
Sp = ws+'/co_assembly/Spades_assembly'
#!mkdir $Sp
for read in reads:
  #if read in done: continue
  unm1 = na+'/%s_trimmed_1.fq.gz'% read
  unm2 = na+'/%s_trimmed_2.fq.gz'% read
  out  = Sp+'/'+read
  #if os.path.exists(out): continue
  bash(string%(unm1,unm2,out),'%s_Spades'%read,'smp')

In [ ]:
# move contigs to the new directory with a report
!mkdir contigs
for read in reads:
  oldass = 'co_assembly/Spades_assembly/%s/contigs.fasta'%read
  newass = 'co_assembly/contigs/%s_contigs.fasta'%read
  !cp $oldass $newass

In [ ]:
# clean Spades directory to save some space
keep = ['contigs.fasta','contigs.paths','scaffolds.fasta','scaffolds.paths','spades.log']
for read in reads:
  for file in os.listdir('Spades_assembly/%s'%read):
    if file not in keep:
      path = 'Spades_assembly/%s/%s'%(read,file)
      !rm -rf $path

# Filter contigs

In [ ]:
reads = ['T0']

In [ ]:
# Filter the contigs base on their size
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=1:00:00
#PBS -l mem=30gb
#PBS -q short
#PBS -S /bin/bash
module load bio/bbtools/38.26
bbduk.sh in=%s out=%s minlen=1000
'''

contigs = ws+'/co_assembly/contigs'
contigs_filtered = ws+'/co_assembly/contigs_filtered'
!mkdir $contigs_filtered
for read in reads:
  #if read in done: continue
  cont = contigs+'/%s_contigs.fasta'% read
  cont_filt = contigs_filtered+'/%s_contigs_filtered.fasta'% read
  if os.path.exists(cont_filt): continue
  bash(string%(cont,cont_filt),'%s_filter'%read,'short')

In [ ]:
# Keep the stats
!mkdir Stats_filtering
!mv *filter.sh.e* Stats_filtering/

# Initial binning

In [ ]:
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=12:00:00
#PBS -l mem=150gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate metawrap-env
metawrap binning -o %s -t 20 -a %s --metabat2 --maxbin2 --concoct %s
'''

contigs_filtered = ws+'/co_assembly/contigs_filtered'
out_bins = ws+'/co_assembly/initial_bin'
clean_reads_path = ws+'/co_assembly/NoAdapt_reads'
!mkdir $out_bins
for read in reads:
    out_folder = out_bins+'/%s_bins' % read
    contigs = contigs_filtered+'/%s_contigs_filtered.fasta' % read
    clean = clean_reads_path+'/%s_trimmed_*.fastq' % read
    #if os.path.exists(out_folder): continue
    bash(string%(out_folder,contigs,clean),'%s_binning'%read,'smp')

In [ ]:
#refine bins
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate metawrap-env
metawrap bin_refinement -o %s -t 20 -A %s -B %s -C %s -c 70 -x 10
'''

bin_refinement = ws+'/bin_refinement'
bins_path = ws+'/initial_bin'
#!mkdir $bin_refinement
for read in reads:
    out_folder = bin_refinement+'/%s_bins_refined_test' % read
    metabat = bins_path+'/%s_bins/metabat2_bins' % read
    maxbin = bins_path+'/%s_bins/maxbin2_bins' % read
    concoct = bins_path+'/%s_bins/semibin_bins/output_bins' % read
    #if os.path.exists(out_folder): continue
    bash(string%(out_folder,metabat,maxbin,concoct),'%s_refinement'%read,'short')

In [ ]:
#move bins to a new folder
!mkdir final_bins
for read in reads:
  oldass = 'bin_refinement/%s_bins_refined/metawrap_70_10_bins/*'%read
  newass = 'final_bins/'
  !cp $oldass $newass

In [ ]:
#bin reassembly
string = '''
#PBS -l nodes=1:ppn=25
#PBS -l walltime=48:00:00
#PBS -l mem=250gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate metawrap-env
metawrap reassemble_bins -o %s -1 %s -2 %s -t 25 -m 250 -c 70 -x 10 -b %s
'''

!mkdir $reassemble_bins
out_folder = ws+'/reassemble_bins'
read1 = ws+'/NoAdapt_reads/all_reads_1.fastq'
read2 = ws+'/NoAdapt_reads/all_reads_2.fastq'
bin_refinement = ws+'/final_bins/'
#if os.path.exists(out_folder): continue
bash(string%(out_folder,read1,read2,bin_refinement),'%s_reassembly'%read,'smp')


In [ ]:
#mapping reads for SemiBin
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=12:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

module load bio/bowtie2/2.4.1

bowtie2-build -f %s %s
bowtie2 -q --fr -x %s -1 %s -2 %s -S %s -p 64

module load bio/samtools/1.9

samtools view -h -b -S %s -o %s -@ 64
samtools view -b -F 4 %s -o %s -@ 64
samtools sort -m 1000000000 %s -o %s -@ 64
samtools index %s
'''
!mkdir initial_bin/semibin
for read in reads:
    contig_1= ws+'/contigs_filtered/%s_contigs_filtered.fasta' %read
    out1 = ws+'/initial_bin/semibin/%s_contigs_filtered.fasta' %read
    read_1 =ws+'/NoAdapt_reads/%s_trimmed_1.fastq' %read
    read_2 =ws+'/NoAdapt_reads/%s_trimmed_2.fastq' %read
    out_sam =ws+'/initial_bin/semibin/%s.sam' %read
    out_bam=ws+'/initial_bin/semibin/%s.bam' %read
    mapped_bam=ws+'/initial_bin/semibin/%s.mapped.bam' %read
    mapped_sorted= ws+'/initial_bin/semibin/%s.mapped.sorted.bam' %read

    #if os.path.exists(out_folder): continue
    bash(string%(contig_1, out1, out1, read_1, read_2, out_sam, out_sam, out_bam, out_bam, mapped_bam, mapped_bam, mapped_sorted,mapped_sorted),'%s_bowtie'%read,'short')



In [ ]:
#Bining with SemiBin. this job use GPUs
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=48:00:00
#PBS -l mem=80gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate SemiBin

SemiBin single_easy_bin -i %s -b %s -o %s -t 16 -r /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda

conda deactivate
'''
for read in reads:
    contig= ws+'/contigs_filtered/%s_contigs_filtered.fasta' %read
    bam_file=ws+'/initial_bin/semibin/%s.mapped.sorted.bam' %read
    out_folder=ws+'/initial_bin/%s_bins/semibin_bins' %read
    bash(string%(contig, bam_file,out_folder),'%s_semibin'%read,'short')


# Bins QC

In [ ]:
#check genomes QC, split genomes in 1000
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=72:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate checkm
checkm lineage_wf -t 20 -x fa %s %s
conda deactivate
'''
!mkdir checkM
genomes_path = ws+'/final_bins'
out_file = ws+'/checkM'
bash(string%(genomes_path, out_file),'checkM','smp')

# Dereplicate Genomes

In [ ]:
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=4:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate drep
dRep dereplicate -d -p 12 %s -g %s --genomeInfo %s -comp 70 -con 10
'''
!mkdir drep
drep_out = ws+'/drep'
genomes_path = ws+'/final_bins/*.fa'
checkm = ws+'/checkM/bin_stats_ext_clean.csv'
bash(string%(drep_out, genomes_path, checkm), 'drep', 'short')

# Taxonomical annotation (GTDB-tk)

In [ ]:
string ='''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=6:00:00
#PBS -l mem=150gb
#PBS -q smp
#PBS -S /bin/bash
# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate gtdbtk
# Run
gtdbtk classify_wf --genome_dir %s --extension fa --out_dir %s --cpus 20
'''
path_genomes = ws+'/drep/dereplicated_genomes/'
out_dir = ws+'/gtdb_classify_drep'
!mkdir $out_dir
bash(string%(path_genomes, out_dir), 'gtdbtk_classify', 'smp')

# Make a tree

In [ ]:
#config file fot Phyloplan
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=01:00:00
#PBS -l mem=10gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate phylophlan

phylophlan_write_config_file -o %s \
    -d a \
    --db_aa diamond \
    --map_aa diamond \
    --msa mafft \
    --trim trimal \
    --tree1 fasttree \
    --tree2 raxml

conda deactivate
'''
out_file = ws+'/custom_config_nt.cfg'
bash(string%(out_file),'phylophlan_custom','short')

In [ ]:
#run Phylophlan
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate phylophlan

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/phylophlan


python /beegfs/work/workspace/ws/ho_kezau83-conda-0/phylophlan/phylophlan/phylophlan.py -i %s \
    --output_folder %s \
    --genome_extension .fasta \
    -d phylophlan \
    -t a \
    --diversity medium \
    -f %s \
    --nproc 20 \
'''
!mkdir phylophlan
genomes = ws+'/padloc/proteomes/'
out_file = ws+'/phylophlan'
config = ws+'/custom_config_nt.cfg'
bash(string%(genomes, out_file, config),'phylophlan','smp')

# MAGs coverage

In [ ]:
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate coverm

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0

# Run
TMPDIR=. coverm genome -t 14 -c %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s \
-d %s -x fa -o coverM_output.tsv
'''
read1_1 = ws+'/NoAdapt_reads/T0A_trimmed_1.fastq'
read1_2 = ws+'/NoAdapt_reads/T0A_trimmed_2.fastq'
read2_1 = ws+'/NoAdapt_reads/T0B_trimmed_1.fastq'
read2_2 = ws+'/NoAdapt_reads/T0B_trimmed_2.fastq'
read3_1 = ws+'/NoAdapt_reads/T0C_trimmed_1.fastq'
read3_2 = ws+'/NoAdapt_reads/T0C_trimmed_2.fastq'
read4_1 = ws+'/NoAdapt_reads/C2A_trimmed_1.fastq'
read4_2 = ws+'/NoAdapt_reads/C2A_trimmed_2.fastq'
read5_1 = ws+'/NoAdapt_reads/C2B_trimmed_1.fastq'
read5_2 = ws+'/NoAdapt_reads/C2B_trimmed_2.fastq'
read6_1 = ws+'/NoAdapt_reads/C2C_trimmed_1.fastq'
read6_2 = ws+'/NoAdapt_reads/C2C_trimmed_2.fastq'
read7_1 = ws+'/NoAdapt_reads/C4A_trimmed_1.fastq'
read7_2 = ws+'/NoAdapt_reads/C4A_trimmed_2.fastq'
read8_1 = ws+'/NoAdapt_reads/C4B_trimmed_1.fastq'
read8_2 = ws+'/NoAdapt_reads/C4B_trimmed_2.fastq'
read9_1 = ws+'/NoAdapt_reads/C4C_trimmed_1.fastq'
read9_2 = ws+'/NoAdapt_reads/C4C_trimmed_2.fastq'
read10_1 = ws+'/NoAdapt_reads/C8A_trimmed_1.fastq'
read10_2 = ws+'/NoAdapt_reads/C8A_trimmed_2.fastq'
read11_1 = ws+'/NoAdapt_reads/C8B_trimmed_1.fastq'
read11_2 = ws+'/NoAdapt_reads/C8B_trimmed_2.fastq'
read12_1 = ws+'/NoAdapt_reads/C8C_trimmed_1.fastq'
read12_2 = ws+'/NoAdapt_reads/C8C_trimmed_2.fastq'
read13_1 = ws+'/NoAdapt_reads/C15A_trimmed_1.fastq'
read13_2 = ws+'/NoAdapt_reads/C15A_trimmed_2.fastq'
read14_1 = ws+'/NoAdapt_reads/C15B_trimmed_1.fastq'
read14_2 = ws+'/NoAdapt_reads/C15B_trimmed_2.fastq'
read15_1 = ws+'/NoAdapt_reads/C15C_trimmed_1.fastq'
read15_2 = ws+'/NoAdapt_reads/C15C_trimmed_2.fastq'
read16_1 = ws+'/NoAdapt_reads/C40A_trimmed_1.fastq'
read16_2 = ws+'/NoAdapt_reads/C40A_trimmed_2.fastq'
read17_1 = ws+'/NoAdapt_reads/C40B_trimmed_1.fastq'
read17_2 = ws+'/NoAdapt_reads/C40B_trimmed_2.fastq'
read18_1 = ws+'/NoAdapt_reads/C40C_trimmed_1.fastq'
read18_2 = ws+'/NoAdapt_reads/C40C_trimmed_2.fastq'

genome_path = ws+'/drep/dereplicated_genomes'

bash(string%(read1_1,read1_2,read2_1,read2_2,read3_1,read3_2,read4_1,read4_2,read5_1,read5_2,read6_1,read6_2,read7_1,read7_2,read8_1,read8_2,read9_1,read9_2,read10_1,read10_2,read11_1,read11_2,read12_1,read12_2,read13_1,read13_2,read14_1,read14_2,read15_1,read15_2,read16_1,read16_2,read17_1,read17_2,read18_1,read18_2,genome_path), 'coverM', 'short')

# Metabolic prediction

In [ ]:
# A script for running metabolic needs to be created first
# Genomes shoudl have .fasta extension
def metabolic(string,name):
  script = "%s.sh"%name                 #script to write a command
  with open(script, "w") as text_file:  #create and open file
    text_file.write(string)             #write a command into the file 
  !chmod +x $script                     #make it executable

metabolic_script = '''
#!/bin/bash
. /opt/miniconda/etc/profile.d/conda.sh
conda activate metabolic

# Run
perl /opt/METABOLIC/METABOLIC-C.pl -in-gn %s -o %s -r %s

'''
genomes_folder = ws+'/drep/dereplicated_genomes'
path_output = ws+'/Metabolic_results_drep'
path_files = ws+ '/path_files_clean.txt'
!mkdir $path_output
metabolic(metabolic_script%(genomes_folder,path_output, path_files),'metabolic_script')

In [ ]:
# The previous created script is run via singularity
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=4:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

singularity exec /beegfs/work/workspace/ws/ho_kezau83-conda-0/metabolic/metabolic-4.0.sif \
/bin/bash \
%s
'''
script_path = ws+'/metabolic_script.sh'
bash(string%(script_path),'metabolic_prediction','smp')

In [9]:
# annotation with DRAM
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM.py annotate -i '/beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/drep/dereplicated_genomes/*.fasta' -o %s
'''

out_folder= ws+'/dram_out_bins/'
bash(string%(out_folder), 'dram', 'smp')

10717912


In [5]:
#distill functions with DRAM
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM.py distill -i %s -o %s --trna_path %s --rrna_path %s 
'''
in_file = ws+'/dram_out_bins/annotations.tsv'
out_folder= ws+'/dram_out_bins/distilled'
trna =  ws+'/dram_out_bins/trnas.tsv'
rrna = ws+'/dram_out_bins/rrnas.tsv'
bash(string%(in_file, out_folder, trna,rrna), 'dram-distilled', 'smp')

10721136


# Defence system search

In [ ]:
genomes = sorted(list(set([f.split('.fa')[0] for f in os.listdir('final_bins/')])))
genomes


In [ ]:
#Use PADLOC to predict ds in the MAGs
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=08:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate padloc

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0/padloc_no_drep
padloc --fna %s --cpu 10
'''
!mkdir padloc_no_drep
for genome in genomes:
    path_genomes_fasta = ws+'/final_bins/%s.fa' %genome
    bash(string%(path_genomes_fasta),'%s_padloc'%genome,'short')

In [ ]:
#zip al results in csv format
!zip padloc_result padloc/*.csv

# CRISPR-cas detection

In [ ]:
#make a list of all MAGs
bin_name = sorted(list(set([f.split('.fa')[0] for f in os.listdir('final_bins')])))
bin_name

In [ ]:
# add bin name to contigs
for bin in bin_name:
    file = ws+'/final_bins/%s.fa' % bin
    out = ws+'/final_bins/bins_labeled/%s.fa' % bin
    header = r"s/>/>'{}'_/g".format(bin) 
    !sed $'{header}' $file > $out 

In [ ]:
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=2:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate cctyper

cctyper %s %s --prodigal meta --no_plot -t 10

conda deactivate
'''
!mkdir cctyper
for bin in bin_name:
    path_genomes_fasta = ws+'/final_bins/bins_labeled/%s.fa' %bin
    out_path = ws+'/cctyper/%s' %bin
    bash(string%(path_genomes_fasta, out_path, ),'%s_cctyper'%bin,'short')

In [ ]:
# move sapcer to the new directory 
!mkdir spacers
for bin in bin_name:
  oldass = ws+'/cctyper/%s/spacers/*' %bin
  newass = ws+'/spacers/'
  !cp $oldass $newass

In [ ]:
# move CRISPR-Cas-loci to the new directory 
!mkdir crispr_cas_loci
for genome in genomes:
  oldass = ws+'/cctyper/%s/CRISPR_Cas.tab' %genome
  newass = ws+'/crispr_cas_loci/%s_CRISPR_Cas.tab' %genome
  !cp $oldass $newass

In [ ]:
reads

In [ ]:
!qselect -u ho_kezau83 -s R | xargs qdel


# Phage detection

In [ ]:
# add metagenome names in the headers of contigs for the hungate collection
for read in reads:
    file = ws+'/contigs_filtered/%s_contigs_filtered.fasta' % read
    out = ws+'/contigs_rename/%s_contigs.fasta' % read
    header = r"s/>/>'{}'_/g".format(read) 
    !sed $'{header}' $file > $out 

In [ ]:
#virsoter2
#
string = '''
#PBS -l nodes=1:ppn=30
#PBS -l walltime=1:00:00:00
#PBS -l mem=300gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vs2

#run program
virsorter run --keep-original-seq -i %s -w %s --min-length 5000 -j 30 all
'''
in_file = ws+'/contigs_rename/all_contigs.fasta'
out_file = ws+'/vs2_out'
!mkdir $out_file
bash(string%(in_file, out_file), 'vs2', 'smp')

In [ ]:
#vibrant
string = '''
#PBS -l nodes=1:ppn=30
#PBS -l walltime=12:00:00
#PBS -l mem=300gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vibrant

#run program
VIBRANT_run.py -t 30 -i %s  -f nucl -folder %s -l 9999
'''
in_file = ws+'/contigs_rename/all_contigs.fasta'
out_file = ws+'/vibrant_out_9999'
!mkdir $out_file
bash(string%(in_file, out_file), 'vibrant', 'smp')

# dereplicate phages

In [ ]:
string = '''
#PBS -l nodes=1:ppn=25
#PBS -l walltime=5:00:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate drep
dRep dereplicate -d -p 25 %s -g %s --S_algorithm ANImf -nc .5 -l 5000 -N50W 0 -sizeW 1 --ignoreGenomeQuality --clusterAlg single
'''

!mkdir drep_phages
drep_out = ws+'/drep_phages'
genomes_path = ws+'/phages/*.fasta'
bash(string%(drep_out, genomes_path), 'drep', 'smp')

In [ ]:
#checkV
#clean the viral contigs
string = '''
#PBS -l nodes=1:ppn=25
#PBS -l walltime=72:00:00
#PBS -l mem=250gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate checkv

checkv end_to_end %s %s -t 25 -d %s

conda deactivate
'''
in_file = ws+'/drep_phages/dereplicated_genomes/drep_all_phages.fasta'
db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/checkv_db/checkv-db-v1.0'
out_file = ws+'/checkv'
!mkdir $out_file
bash(string%(in_file, out_file, db), 'checkv', 'smp')

In [ ]:
#merge provirus and virus
!cat checkv/proviruses.fna checkv/viruses.fna > checkv/all_virus.fna

In [ ]:
#filter phages by qc and length. Filtering of ID was done in R 
!cat checkv/all_virus.fna | grep -A 1 -f checkv/phages_id_10000kb.txt > checkv/filtered_phages.fasta

In [ ]:
!cat checkv/filtered_phages.fasta | sed 's/--//g' > checkv/filtered_phages_clean.fasta

In [ ]:
!cat checkv/filtered_phages.fasta | grep ">" | wc

# Match phages host

In [ ]:
#run gtdb_tk to create files to add MAGs to Iphop db
string ='''
#PBS -l nodes=1:ppn=25
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -q smp
#PBS -S /bin/bash
# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate gtdbtk
# Run
gtdbtk de_novo_wf --bacteria --outgroup_taxon p__Patescibacteria --genome_dir %s --extension fa --out_dir %s --cpus 20 --force
'''
path_genomes = ws+'/final_bins/'
out_dir = ws+'/gtdb_classify_iphop'
!mkdir $out_dir
bash(string%(path_genomes, out_dir), 'gtdbtk_classify', 'smp')

In [ ]:
#add MAGs to the Iphop db
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=48:00:00
#PBS -l mem=300gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate iphop_env

iphop add_to_db --fna_dir %s --gtdb_dir %s --out_dir %s --db_dir %s -t 20
'''
mags = ws+'/final_bins/'
gtdb = ws+'/gtdb_classify_iphop/infer/'
out_dir = ws+'/Sept_2021_pub_w_silage'
iphop_db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/iphop_db/Sept_2021_pub'
bash(string%(mags, gtdb,out_dir, iphop_db), 'iphop_add_db', 'smp')

In [ ]:
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=48:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate iphop_env

# Run
iphop  predict -t 20 --fa_file %s --db_dir %s --out_dir %s

'''
file = ws+'/checkv/filtered_phages_clean.fasta'
db = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/iphop_db/Sept_2021_pub'
out_dir = ws+ '/iphop_output'
!mkdir $out_dir
bash(string%(file, db, out_dir), 'iphop', 'smp')

In [ ]:
!cat iphop_output/filtered_phages_clean_clean.fna | grep ">" | wc

# Annotate phages

In [ ]:
#prodigal
#predict genes and proteins
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=2:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate padloc

prodigal -i %s -o %s -a %s  -p meta

conda deactivate
'''
in_file = ws+'/checkv/filtered_phages_clean.fasta'
outfile1 = ws+'/checkv/filtered_phages_clean.gbk'
outfile2 = ws+'/checkv/filtered_phages_clean.faa'
bash(string%(in_file, outfile1, outfile2), 'prodigal', 'short')

In [ ]:
#vcontact2
#create a map genes to genomes
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=2:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vContact2

vcontact2_gene2genome -p %s -o %s -s 'Prodigal-FAA'

conda deactivate
'''

infile = ws+'/checkv/filtered_phages_clean.faa'
outfile = ws+'/checkv/phages_genomes_g2g.csv'
bash(string%(infile, outfile), 'vcontact2', 'short')

In [ ]:
#vcontact2
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=48:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

module load devel/java_jdk/11.0.4
#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vContact2

vcontact2 --raw-proteins %s \
--proteins-fp %s \
--db 'None' \
--pcs-mode MCL \
--vcs-mode ClusterONE \
--c1-bin /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/envs/vContact2/bin/cluster_one-1.0.jar \
--output-dir %s \
-t 20

conda deactivate
'''
infile = ws+'/inphared/inphared_silage_phages.faa'
g2g = ws+'/inphared/gene_to_genome.csv'
outfile = ws+'/vcontact2_inphared'
!mkdir $outfile
bash(string%(infile, g2g, outfile), 'vcontact2_anno', 'smp')

In [ ]:
#taxonomy with PhageGCN
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate PhaGCN

python /beegfs/work/workspace/ws/ho_kezau83-conda-0/PhaGCN/run_Speed_up.py --contigs %s --len 8000

conda deactivate
'''

infile = ws+'/checkv/filtered_phages_clean.fasta'

bash(string%(infile), 'PhageGCN', 'short')

# phages coverage

In [ ]:
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate coverm

cd /beegfs/work/workspace/ws/ho_kezau83-silage_timetrial-0

# Run
TMPDIR=. coverm contig -t 14 -c %s %s -r %s -o %s -m rpkm
'''
!mkdir coverm_phages
for read in reads:
    read1 = ws+'/NoAdapt_reads/%s_trimmed_1.fastq' %read
    read2 = ws+'/NoAdapt_reads/%s_trimmed_2.fastq' %read
    phages = ws+'/checkv/filtered_phages_clean.fasta'
    out_file = ws+'/coverm_phages/%s_coverm.tsv' %read
    bash(string%(read1,read2,phages,out_file), '%s_coverM'%read, 'short')

In [ ]:
#compress files
!zip coverm_phages coverm_phages/*

# Viral functional annotation

In [ ]:
#setup DB
string = '''
#PBS -l nodes=1:ppn=5
#PBS -l walltime=24:00:00
#PBS -l mem=300gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

cd /beegfs/work/workspace/ws/ho_kezau83-conda-0/DRAM_data/

DRAM-setup.py prepare_databases --output_dir %s --threads 5
'''

db_path = '/beegfs/work/workspace/ws/ho_kezau83-conda-0/DRAM_data/'
bash(string%(db_path), 'dram_setup', 'smp')

In [ ]:
#fix DB
string = '''
#PBS -l nodes=1:ppn=10
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

# Activate conda enviroment
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

cd /beegfs/work/workspace/ws/ho_kezau83-conda-0/DRAM_data/
DRAM-setup.py update_description_db
'''
bash(string,'dram_fix', 'short')

In [ ]:
#Use Virsorter to match DRAM format of viral contigs

string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=1:00:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vs2

#run program
virsorter run --seqname-suffix-off --viral-gene-enrich-off --prep-for-dramv -i %s -w %s -j 14 all
'''
in_file = ws+'/checkv/filtered_phages_clean.fasta'
out_file = ws+'/vs2_dram'
!mkdir $out_file
bash(string%(in_file, out_file), 'vs2_to_dram', 'short')

In [13]:
#Annotate with DRAM
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM-v.py annotate -i %s -v %s -o %s
'''

in_file = ws+'/vs2_dram/for-dramv/final-viral-combined-for-dramv.fa'
in_virsorter = ws+'/vs2_dram/for-dramv/viral-affi-contigs-for-dramv.tab'
out_folder= ws+'/dram_out'
bash(string%(in_file, in_virsorter, out_folder), 'dram-v', 'smp')

10714236


In [14]:
#distill functions with DRAM

string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate DRAM

#run program
DRAM-v.py distill -i %s -o %s
'''
in_file = ws+'/dram_out/annotations.tsv'
out_folder= ws+'/dram_out/distilled'
bash(string%(in_file, out_folder), 'dram-distilled', 'smp')


10714237


# Active prophages

In [ ]:
#make a list of the drep MAGs
bin_name = sorted(list(set([f.split('.fasta')[0] for f in os.listdir('drep/dereplicated_genomes/fasta')])))
bin_name

In [ ]:
# add bin name to contigs
for bin in bin_name:
    file = ws+'/drep/dereplicated_genomes/fasta/%s.fasta' % bin
    out = ws+'/drep/dereplicated_genomes/%s.fasta' % bin
    header = r"s/>/>'{}'_/g".format(bin) 
    !sed $'{header}' $file > $out 

In [ ]:
#merge bin contigs in one file
!cat drep/dereplicated_genomes/*.fasta  > all_bin_contigs.fasta

In [ ]:
#find prophages with vibrant
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=24:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate vibrant

#run program
VIBRANT_run.py -t 14 -i %s  -f nucl -folder %s -l 9999
'''
in_file = ws+'/all_bin_contigs.fasta'
out_file = ws+'/vibrant_propagate'
!mkdir $out_file
bash(string%(in_file, out_file), 'vibrant', 'short')

In [ ]:
#run PropagATE
string = '''
#PBS -l nodes=1:ppn=20
#PBS -l walltime=24:00:00
#PBS -l mem=200gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate propagate

#run program
Propagate -f %s -r %s %s -v %s -o %s --clean -t 20
'''
prophages_file = ws+'/contigs_rename/all_contigs.fasta'
reads_1 = ws+'/NoAdapt_reads/C2A_trimmed_1.fastq'
reads_2 = ws+'/NoAdapt_reads/C2A_trimmed_2.fastq'
vibrant_file = ws+'/vibrant_out_9999/VIBRANT_all_contigs/VIBRANT_results_all_contigs/VIBRANT_integrated_prophage_coordinates_all_contigs.tsv'
out_folder = ws+'/PropagAtE_out_9999_c2'
!mkdir $out_file
bash(string%(prophages_file, reads_1, reads_2, vibrant_file, out_folder), 'propagATE', 'smp')

# Match sapcers wit vOTU

In [ ]:
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=24:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
module load bio/blastplus/2.5.0
makeblastdb -in %s -out %s  -parse_seqids -dbtype nucl
'''
!mkdir phages_drep_db
file_path = ws+'/checkv/filtered_phages_clean.fasta'
db_path = ws+'/phages_drep_db/phages_drep_db'
bash(string%(file_path, db_path),'makedb', 'short')

In [ ]:
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=24:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
module load bio/blastplus/2.5.0
blastn -task blastn-short -query %s -db %s -out %s -perc_identity 95 -max_target_seqs 1 -num_threads 16 -evalue 1e-4 -outfmt 6 

'''
file_path = ws+'/all_sapcers.fasta'
db_path = ws+'/phages_drep_db/phages_drep_db'
out_file = ws+'/spacers_match_phages.txt'
bash(string%(file_path, db_path,out_file),'blastn', 'short')

# VICTOR tree

In [ ]:
!cp checkv/filtered_phages_clean.fasta checkv/victor/

In [ ]:
!awk '/^>/ { file=substr($1,2) ".fasta" } { print > file }' *.fasta

In [ ]:
#make a list of the drep phages
phage_name = sorted(list(set([f.split('.fasta')[0] for f in os.listdir('checkv/victor/')])))
phage_name

In [ ]:
#prodigal
#predict genes and proteins
string = '''
#PBS -l nodes=1:ppn=16
#PBS -l walltime=2:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash

#activate conda
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate padloc

prodigal -i %s -o %s -a %s -p meta

conda deactivate
'''
for phage in phage_name:
    in_file = ws+'/checkv/victor/%s.fasta' %phage
    outfile1 = ws+'/checkv/victor/%s.gbk' %phage
    outfile2 = ws+'/checkv/victor/%s.faa' %phage
    bash(string%(in_file, outfile1, outfile2), '%s_prodigal'%phage, 'short')

In [ ]:
!zip phage_proteomes checkv/victor/*.faa

In [ ]:
!cat spacers/* > all_sapcers.fasta

In [ ]:
!qdel 10685701

In [ ]:
!gunzip -d co_assembly/NoAdapt_reads/*

In [ ]:
!rename .fq .fastq co_assembly/NoAdapt_reads/*

In [ ]:
!rename bin. C40C_bin. bin_refinement/C40C_bins_refined/metawrap_70_10_bins/*

In [ ]:
!sed "s/bin.[0-9]*/&.fa/g" checkM/bin_stats_ext.csv > checkM/bin_stats_ext_clean.csv

In [12]:
rm -r dram_out/

In [ ]:
!qstat -u ho_kezau83

In [ ]:
rm -r Sept_2021_pub_w_silage/

In [ ]:
!cat contigs_rename/*.fasta > contigs_rename/all_contigs.fasta

In [ ]:
!mv NoAdapt_reads/reads_2/*_2.fastq NoAdapt_reads

In [ ]:
!zip all_genomes drep/dereplicated_genomes/* 

In [ ]:
!cat drep_phages/dereplicated_genomes/drep_all_phages.fasta | grep ">" | wc

In [ ]:
!mkdir drep/dereplicated_genomes/fasta

In [ ]:
!mv drep/dereplicated_genomes/*.fasta drep/dereplicated_genomes/fasta/